## 1. 라이브러리 및 CBraMod 설치

In [1]:
!pip install mne torch pandas tqdm einops

In [2]:
!git clone https://github.com/wjq-learning/CBraMod.git
!pip install -r CBraMod/requirements.txt

fatal: destination path 'CBraMod' already exists and is not an empty directory.


In [16]:
import os
import sys
import numpy as np
import pandas as pd
import mne
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from glob import glob
import warnings
from einops import rearrange
from einops.layers.torch import Rearrange

# MNE 경고 무시
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- 경로 및 하이퍼파라미터 설정 ---

# ❗️❗️ 사용자의 환경에 맞게 이 부분의 경로를 반드시 수정해주세요. ❗️❗️
# 1. 전처리된 EEG 데이터가 있는 폴더 경로 (예: "HBN_EEG_Preprocessed")
PREPROCESSED_ROOT = "/data5/open_data/HBN/Preprocessed_EEG/0922try_bySubject/" 
# 2. participants.tsv 파일이 있는 원본 BIDS 폴더 경로 (예: "/data/HBN/EEG_BIDS")
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
# 3. CBraMod 프로젝트를 클론한 경로
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"
# 4. 다운로드한 사전 학습 가중치 파일 경로
PRETRAINED_MODEL_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth" 

# CBraMod 모델 코드를 불러오기 위해 경로 추가
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod

# 모델 하이퍼파라미터 (데이터를 모델 아키텍처에 맞춤)
N_CHANNELS = 128
N_TIMES = 400           # 세그먼트 길이 (200 * 2)
PATCH_SIZE = 200        # 모델이 기대하는 패치 크기 (in_dim)
SEQ_LEN = N_TIMES // PATCH_SIZE # 패치 개수 (seq_len)

# 학습 하이퍼파라미터
BATCH_SIZE = 64
FINETUNE_EPOCHS = 5
LR = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
N_OUTPUTS = 4

print(f"Using device: {DEVICE}")

Using device: cuda


In [17]:
def collect_preprocessed_files(root_path):
    """지정된 폴더에서 전처리된 모든 EEG 파일(*_eeg_pp.set) 경로를 수집합니다."""
    print(f"\nCollecting preprocessed EEG files from: {root_path}")
    files = glob(os.path.join(root_path, "sub-*", "*_eeg_pp.set"), recursive=True)
    return [(f, "") for f in files]

def load_all_participants(bids_root, datasets):
    """지정된 데이터셋 목록에서 participants.tsv를 병합합니다."""
    all_participants = []
    required_cols = ['participant_id', 'p_factor', 'internalizing', 'externalizing', 'attention']
    
    print(f"Loading participant labels from: {datasets}")
    for ds in tqdm(datasets, desc="Loading participants.tsv"):
        participants_file = os.path.join(bids_root, ds, "participants.tsv")
        
        if not os.path.exists(participants_file):
            continue
        try:
            df = pd.read_csv(participants_file, sep='\t')
            if not all(col in df.columns for col in required_cols):
                continue
            df_filtered = df[required_cols]
            all_participants.append(df_filtered)
        except Exception as e:
            print(f"Error reading {participants_file}: {e}")
            continue
            
    if all_participants:
        combined_df = pd.concat(all_participants, ignore_index=True)
        combined_df['participant_id'] = combined_df['participant_id'].str.strip()
        final_df = combined_df.drop_duplicates(subset=['participant_id']).dropna(subset=required_cols).reset_index(drop=True)
        return final_df
    return pd.DataFrame(columns=required_cols)

In [18]:
class EEGDataset(Dataset):
    """ 이미 전처리된 .set 파일을 빠르게 읽는 데이터셋 (캐싱 적용) """
    def __init__(self, file_label_pairs, labels_df, segment_len=N_TIMES):
        self.file_label_pairs = file_label_pairs
        self.labels_df = labels_df.set_index("participant_id")
        self.segment_len = segment_len
        self.sample_indices = []

        # 새로 추가: MNE raw 객체를 저장할 캐시 딕셔너리
        # DataLoader의 각 워커 프로세스별로 독립적인 캐시가 유지됩니다.
        self._raw_cache = {} 
        
        # tqdm 대신, 전역 변수 N_CHANNELS를 사용했다고 가정합니다.
        # (원래 코드에서 N_CHANNELS는 128로 설정되어 있습니다.)
        N_CHANNELS = 128 

        for idx, (eeg_file, _) in enumerate(tqdm(file_label_pairs, desc="Indexing preprocessed files")):
            subj_match = os.path.basename(eeg_file).split("sub-")
            if len(subj_match) < 2: continue
            
            subj = subj_match[1].split("_")[0]
            subj_id = f"sub-{subj}"
            
            if subj_id not in self.labels_df.index:
                continue
            
            try:
                # 인덱싱 단계에서는 preload=False로 유지하여 메타데이터만 빠르게 로드
                raw = mne.io.read_raw_eeglab(eeg_file, preload=False, verbose=False)

                if len(raw.ch_names) != N_CHANNELS:
                    continue
                
                num_segments = raw.n_times // segment_len
                
                for i in range(num_segments):
                    self.sample_indices.append((idx, i * segment_len, subj_id))
            except Exception as e:
                # print(f"Error indexing {eeg_file}: {e}")
                continue

    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
    file_idx, start, subj = self.sample_indices[idx]
    eeg_file, _ = self.file_label_pairs[file_idx]

    # 부분 로드 (전체 preload 금지)
    raw = mne.io.read_raw_eeglab(eeg_file, preload=False, verbose=False)
    seg = raw.get_data(start=start, stop=start+self.segment_len)

    if seg.shape[1] != self.segment_len:
        padding = np.zeros((seg.shape[0], self.segment_len - seg.shape[1]))
        seg = np.concatenate([seg, padding], axis=1)

    label = self.labels_df.loc[subj].values.astype("float32")

    x = torch.tensor(seg, dtype=torch.float32)
    y = torch.tensor(label, dtype=torch.float32)
    return x, y


IndentationError: expected an indented block after function definition on line 45 (2219537447.py, line 46)

In [19]:
class EEGRegressor(nn.Module):
    """ 사전 학습된 CBraMod 인코더 위에 회귀 헤드를 추가한 모델 """
    def __init__(self, encoder, n_outputs=N_OUTPUTS):
        super().__init__()
        self.encoder = encoder
        self.encoder.proj_out = nn.Identity()
        
        with torch.no_grad():
            dummy_input = torch.randn(1, N_CHANNELS, SEQ_LEN, PATCH_SIZE)
            dummy_output = self.encoder(dummy_input)
            flattened_size = dummy_output.view(1, -1).shape[1]

        self.reg_head = nn.Sequential(
            Rearrange('b c s p -> b (c s p)'),
            nn.Linear(flattened_size, 512),
            nn.ELU(),
            nn.Dropout(0.1),
            nn.Linear(512, n_outputs)
        )

    def forward(self, x):
        # 입력 x: (B, C, 400) -> (B, C, 2, 200)
        x = rearrange(x, 'b c (s p) -> b c s p', s=SEQ_LEN, p=PATCH_SIZE)
        features = self.encoder(x)
        return self.reg_head(features)

In [20]:
import gc

def finetune(model, train_loader, val_loader, epochs, lr, device):
    print("\n[STEP 5] Starting Fine-tuning...")
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss() 
    
    best_val_loss = float('inf')

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        total_train_batches = len(train_loader)
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")

        for i, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss = criterion(preds, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

            if (i + 1) % 10 == 0 or (i + 1) == total_train_batches:
                running_avg_loss = train_loss / (i + 1)
                print(f"\r[Train] Batch {i + 1}/{total_train_batches} | Loss: {loss.item():.4f} | Avg Loss: {running_avg_loss:.4f}", end="")
        
        print() 

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                preds = model(X)
                val_loss += criterion(preds, y).item()
        
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0
        print(f"[Summary] Train Avg Loss: {avg_train_loss:.4f}, Val Avg Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            print(f"🎉 New best model found! Saving model to 'best_finetuned_model.pth'...")
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_finetuned_model.pth')
    
    torch.cuda.empty_cache()
    
    gc.collect()

    print("\n✅ Fine-tuning complete!")
    return model

In [21]:
# --- 데이터셋 그룹 정의 ---
train_datasets = [f"ds00{i}" for i in range(5505, 5517) if i != 5509]
val_datasets = ["ds005509"]

# --- 훈련/검증용 라벨 로드 ---
df_train_participants = load_all_participants(BIDS_ROOT, train_datasets)
df_val_participants = load_all_participants(BIDS_ROOT, val_datasets)

# --- 전처리된 EEG 파일 수집 및 분할 ---
all_pp_files = collect_preprocessed_files(PREPROCESSED_ROOT)
train_subjects_set = set(df_train_participants['participant_id'])
val_subjects_set = set(df_val_participants['participant_id'])
train_files, val_files = [], []

for f, e in tqdm(all_pp_files, desc="Assigning files to train/val sets"):
    try:
        subj_id = "sub-" + os.path.basename(f).split("sub-")[1].split("_")[0]
        if subj_id in train_subjects_set:
            train_files.append((f, e))
        elif subj_id in val_subjects_set:
            val_files.append((f, e))
    except IndexError:
        continue

# --- 데이터셋 및 데이터 로더 생성 ---
print("\nCreating Datasets and DataLoaders...")
train_dataset = EEGDataset(train_files, df_train_participants, segment_len=N_TIMES)
val_dataset = EEGDataset(val_files, df_val_participants, segment_len=N_TIMES)

# tqdm 진행률 표시가 안 될 경우 num_workers=0으로 변경
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # 👈 멀티프로세스 I/O 제거
    pin_memory=False,  # 👈 GPU로 미리 올리지 않음
    persistent_workers=False
)


val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,          # 검증 데이터는 순서 유지
    num_workers=0,
    pin_memory=False
)

# --- 사전 학습 모델 로딩 및 최종 모델 구성 ---
print("\nLoading pre-trained model...")
cbra_mod_base = CBraMod(in_dim=PATCH_SIZE, seq_len=SEQ_LEN)

if os.path.exists(PRETRAINED_MODEL_PATH):
    state_dict = torch.load(PRETRAINED_MODEL_PATH, map_location=DEVICE)
    cbra_mod_base.load_state_dict(state_dict, strict=False)
    print("Pre-trained weights loaded successfully!")
else:
    print(f"Warning: Pre-trained weights not found at '{PRETRAINED_MODEL_PATH}'. Training from scratch.")

regressor = EEGRegressor(encoder=cbra_mod_base, n_outputs=N_OUTPUTS)



Loading participant labels from: ['ds005505', 'ds005506', 'ds005507', 'ds005508', 'ds005510', 'ds005511', 'ds005512', 'ds005513', 'ds005514', 'ds005515', 'ds005516']


Loading participants.tsv: 100%|██████████| 11/11 [00:00<00:00, 26.27it/s]


Loading participant labels from: ['ds005509']


Loading participants.tsv: 100%|██████████| 1/1 [00:00<00:00, 21.18it/s]


Assigning files to train/val sets: 100%|██████████| 25944/25944 [00:00<00:00, 373496.29it/s]



Creating Datasets and DataLoaders...


Indexing preprocessed files:   0%|          | 8/22425 [00:01<37:09, 10.06it/s]  /home/RA/.local/lib/python3.10/site-packages/pymatreader/utils.py:296: UserWarning: pymatreader cannot import Matlab string variables. Please convert these variables to char arrays in Matlab.
  warn(
Indexing preprocessed files:   3%|▎         | 781/22425 [01:47<49:41,  7.26it/s]  


KeyboardInterrupt: 

In [ ]:
# --- 모델 학습 실행 ---
finetuned_model = finetune(
    regressor,
    train_loader, 
    val_loader, 
    epochs=FINETUNE_EPOCHS, 
    lr=LR, 
    device=DEVICE
)

print("\n🚀 All steps complete. The best model is saved as 'best_finetuned_model.pth'.")


[STEP 5] Starting Fine-tuning...

--- Epoch 1/20 ---
[Train] Batch 25069/25069 | Loss: 0.6356 | Avg Loss: 0.6653
[Summary] Train Avg Loss: 0.6653, Val Avg Loss: 0.6589
🎉 New best model found! Saving model to 'best_finetuned_model.pth'...

--- Epoch 2/20 ---
[Train] Batch 25069/25069 | Loss: 0.4344 | Avg Loss: 0.6633
[Summary] Train Avg Loss: 0.6633, Val Avg Loss: 0.6595

--- Epoch 3/20 ---
[Train] Batch 25069/25069 | Loss: 0.5959 | Avg Loss: 0.6633
[Summary] Train Avg Loss: 0.6633, Val Avg Loss: 0.6645

--- Epoch 4/20 ---
[Train] Batch 25069/25069 | Loss: 0.5907 | Avg Loss: 0.6632
[Summary] Train Avg Loss: 0.6632, Val Avg Loss: 0.6593

--- Epoch 5/20 ---
[Train] Batch 5740/25069 | Loss: 0.6900 | Avg Loss: 0.6638

KeyboardInterrupt: 

In [57]:
torch.save(regressor.state_dict(), "checkpoint_epoch5.pth")
print("✅ 모델을 안전하게 저장했습니다.")

✅ 모델을 안전하게 저장했습니다.
